# Week 7 Day 1

Fine-tune an open-source model to Predict Product Prices

Please see this notebook in Google Colab:

https://colab.research.google.com/drive/15rqdMTJwK76icPBxNoqhI7Ww8UM-Y7ni?usp=sharing

In [ ]:
!pip install -q bitsandbytes


In [ ]:
!pip install -q pydrive gdown

In [1]:
# imports

import os
import re
import math
import torch
import transformers

from tqdm import tqdm
from dotenv import load_dotenv
from huggingface_hub import login

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, set_seed
from peft import LoraConfig, PeftModel
from datetime import datetime

D:\anaconda3\envs\llms\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 'Could not find module 'D:\anaconda3\envs\llms\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
D:\anaconda3\envs\llms\Lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/731

In [2]:
#environment
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [3]:
# Log in to HuggingFace

hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [4]:
%matplotlib inline

In [5]:
# Constants

BASE_MODEL = "meta-llama/Meta-Llama-3.1-8B"
FINETUNED_MODEL = f"nndk91/ft:gpt-4o-mini-2024-07-18:personal:pricer:AsnYjTRg"

# Hyperparameters for QLoRA Fine-Tuning

LORA_R = 32
LORA_ALPHA = 64
TARGET_MODULES = ["q_proj", "v_proj", "k_proj", "o_proj"]

In [6]:
base_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto")

OSError: [WinError 126] The specified module could not be found. Error loading "D:\anaconda3\envs\llms\Lib\site-packages\intel_extension_for_pytorch\bin\intel-ext-pt-gpu.dll" or one of its dependencies.

In [9]:
print(f"Memory footprint: {base_model.get_memory_footprint() / 1e9:,.1f} GB")

NameError: name 'base_model' is not defined

In [ ]:
#install transformers for torch quantization
#!pip install -q "transformers>=4.45.1"

In [10]:
# Load the Base Model on CPU using 8bit
from transformers import AutoModelForCausalLM
import torch
import torch.quantization
"""
# Load the model onto CPU
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float32,
    low_cpu_mem_usage=True  # This ensures the model is loaded with minimal CPU memory usage
)
"""

# Define a function to process each layer
def process_layer(layer):
    if isinstance(layer, torch.nn.Linear):
        return torch.quantization.quantize_dynamic(
            layer,
            {torch.nn.Linear},
            dtype=torch.qint8
        )
    return layer

In [ ]:
# Load the model onto CPU
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL, 
    torch_dtype=torch.float32, 
    device_map=None
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
import torch
import intel_extension_for_pytorch as ipex

# Set the device to Intel GPU (if available)
device = torch.device("xpu")  # "xpu" is the alias for Intel GPU in IPEX

# Move your model and data to the Intel GPU
model = model.to(device)
input_data = input_data.to(device)

# Now you can run your model on the Intel GPU
output = model(input_data)

In [ ]:
print(f"Device: {device}")
print(f"Is XPU available: {torch.xpu.is_available()}")

In [ ]:
print(f"Memory footprint: {model.get_memory_footprint() / 1e9:,.1f} GB")

In [ ]:
import torch; 
import intel_extension_for_pytorch as ipex; 
print(torch.__version__); 
print(ipex.__version__); 
[print(f'[{i}]: {torch.xpu.get_device_properties(i)}') for i in range(torch.xpu.device_count())];

In [ ]:
!pip install torch==2.5.0 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!pip install intel-extension-for-pytorch==2.5.10+xpu --extra-index-url https://pytorch-extension.intel.com/release-whl/stable/bmg/us/